In [4]:
import cv2
import matplotlib.pyplot as plt
import face_recognition
import glob
import numpy as np
from pathlib import Path
from tqdm import tqdm

In [5]:
#图片剪裁
def image_crop_padding(image,x1,y1,x2,y2):
    h,w,_ = image.shape
    
    crop_x1 = max(0, x1)
    crop_y1 = max(0, y1)
    crop_x2 = min(w - 1, x2 - 1)  
    crop_y2 = min(h - 1, y2 - 1)
    
    #从原图上提取需要被裁剪的区域（可能不是完整正方形）
    roi_image = image[crop_y1: crop_y2 + 1, crop_x1: crop_x2 + 1]
    #创建正方形画布
    dst_image = np.zeros((x2-x1, y2-y1),dtype="uint8" )
    
    #如果出现裁剪出了边缘，则填充边界
    if crop_x1 != x1 or crop_x2 != x2 or crop_y1 != y1 or crop_y2 != y2:     
        up_y = abs(crop_y1 - y1) 
        down_y = abs(crop_y2 - y2)
        left_x = abs(crop_x1 - x1)
        right_x = abs(crop_x2 - x2)
        
        dst_image = cv2.copyMakeBorder(roi_image,up_y,down_y,left_x, right_x,cv2.BORDER_REPLICATE)
    else:
        dst_image = roi_image
    
    return dst_image


In [23]:
def main(folder_path, size_threshold=250, scale=4):
    root_path = Path(folder_path)
    face_path = root_path / "face" 
    face_path.mkdir(exist_ok=True)
    try:
        with tqdm(glob.glob(str(root_path / "*.??[Gg]")), ncols=5) as t:
            for image_path in t:
                image_path = Path(image_path)
                image = face_recognition.load_image_file(image_path)
                #过滤小尺寸图片
                if image.shape[0] < 1500 or image.shape[1] < 1500:
                    continue

                #缩小尺寸因子，缩小图片来检测获取人脸坐标，加快速度，不影响实际保存尺寸
                scale = scale
                rez_img = cv2.resize(image,(0, 0), fx=1/scale, fy=1/scale, interpolation=cv2.INTER_NEAREST)

                face_locations = face_recognition.face_locations(rez_img)
                if len(face_locations) > 0:
                    #找出最大的人脸
                    length_max = x_center = y_center = 0
                    for coordinates in face_locations:
                        y1, x2 ,y2, x1 = coordinates
                        x1 *= scale
                        y1 *= scale
                        x2 *= scale
                        y2 *= scale

                        length = x2-x1
                        if length > length_max:
                            length_max = length
                            x_center = x1 + length//2
                            y_center = y1 + length//2

                    #获得人脸框坐标（全脸）       
        #             print(f"最大的为 {length_max}")
                    x1_new = x_center-int(length_max/1.2)
                    y1_new = y_center-int(length_max/1.2) - int(length_max/5)
                    x2_new = x_center+int(length_max/1.2)
                    y2_new =  y_center+int(length_max/1.2) - int(length_max/5)

                    if length_max > size_threshold:
                        #裁剪图片，自动padding
                        face_image = image_crop_padding(image,x1_new,y1_new,x2_new,y2_new)
                        #保存人脸图片
                        save_path = face_path / (str(length_max) + "_" + image_path.name)
                        face_image = face_image[...,::-1]
                        cv2.imencode('.jpg', face_image)[1].tofile(str(save_path))
                    
#                     #画出人脸框
#                     rect = cv2.rectangle(image.copy(),(x1_new, y1_new ),(x2_new, y2_new),(0,0,255), thickness=10)
#                     plt.figure(figsize=(20,20))
#                     plt.imshow(rect)
#                     plt.show()

#                 elif len(face_locations) == 0:
#                     print("未检测到人脸")
#                     plt.figure(figsize=(20,20))
#                     plt.imshow(image)
#                     plt.show()
    
    #避免意外退出后tqdm后进度条显示异常
    except KeyboardInterrupt:
        t.close()
        raise
    t.close()

In [24]:
folder_path = r"imgs"
main(folder_path, size_threshold=250, scale=4)

100%|█| 10/10 [00:07<00:00,  1.00it/s]


In [ ]:
4-1.04
3-1.51
2-3.00